
<img src='./img/nsidc_logo.png'/>

## NSIDC Valkyrie : Point Cloud Data Access


The Valkyrie project provides web services for ordering spatially and temporally subsetted Lidar point cloud data from the [BLATM L1B](https://nsidc.org/data/BLATM1B), [ILATM L1B v1](https://nsidc.org/data/ilatm1b/versions/1), [ILATM L1B V2](https://nsidc.org/data/ILATM1B), [ILVIS2](https://nsidc.org/data/ILVIS2) and [IceSat GLAH06](https://nsidc.org/data/GLAH06/) data products. The following table describes the temporal and spatial coverage of each of these dataset as well as the sensor and platform used to acquire the data.


---

|              | Spatial Coverage                                                      | Temporal Coverage                              | Platform                                              | Sensor                   |
|--------------|-----------------------------------------------------------------------|------------------------------------------------|-------------------------------------------------------|--------------------------|
| BLATM L1B    | South: N:-53, S: -90, E:180, W:-180 North: N:90, S: 60, E:180, W:-180 | 23 June 1993 - 30 October 2008                 | DC-8, DHC-6, P-3A ORION, P-3B                         | ATM                      |
| ILATM L1B V1 | South: N:-53, S: -90, E:180, W:-180 North: N:90, S: 60, E:180, W:-180 | 31 March 2009 - 8 November 2012 (updated 2013) | AIRCRAFT, DC-8, P-3B                                  | ATM                      |
| ILATM L1B V2 | South: N:-53, S: -90, E:180, W:-180 North: N:90, S: 60, E:180, W:-180 | 20 March 2013 - 16 May 2019 (updated 2020)     | C-130, DC-8, HU-25A, HU-25C, P-3B, WP-3D ORION        | ATM                      |
| ILVIS2       | North: N:90, S: 60, E:180, W:-180                                     | 25 August 2017 - 20 September 2017             | AIRCRAFT, B-200, C-130, DC-8, G-V, HU-25C, P-3B, RQ-4 | ALTIMETERS, LASERS, LVIS |
| GLAH06       | Global: N:86, S: -86, E:180, W:-180                                     |     20 February 2003 - 11 October 2009        | IceSat | ALTIMETERS, CD, GLAS, GPS, GPS Receiver, LA, PC


--- 

> Note: If you have any qustions about the data please contact NSIDC user services at users@nsidc.org

In this tutorial we are going to use iPyLeaflet and other Jupyter widgets to select our constraints and put a data request to our Valkyrie API.

First to get you familiarized with the combined coverage of these products run the next cell and play the Youtube video.




In [ ]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('jRB1OEDXXwY')

## NASA's EarthData Credentials

The first step to start working with Valkyrie data is to login into the NASA's Earth Data system. The output will be shown in the logs window.

We are going to use a user interface to build the parameters we need to post data orders to Valkyrie but this can be done in your own programmatic way, please refer to the Valkyrie documentation or the NSIDC Swagger [OpenAPI documentation](https://staging.nsidc.org/apps/orders/api/).



In [ ]:
# We import our Valkyrie client library
from valkyrie import valkyrie_ui as valkyrie
# We instantiate our interface telling which map projection we want to start.
# the Valid values are: global, south, north
v = valkyrie('north')
# Now we enter our NASA Earth Data Credentials and verify that they work.
# If you are using Jupyter Lab you can see the output in the logs window.
v.render_credentials()

We are going to render the user interface, if you change the hemisphere you need to execute this cell again.
This user interface uses [ipylaflet](https://blog.jupyter.org/interactive-gis-in-jupyter-with-ipyleaflet-52f9657fa7a) which allows us to draw
polygons or bounding boxes for our area selection. We can also edit and delete these geometries using the the widget controls in the map.

The **"Get Granule Count"** button will query [NASA's CMR](https://earthdata.nasa.gov/eosdis/science-system-description/eosdis-components/cmr) to get a granule count for the current parameters, we need to have a geometry and one or more datasets selected.

> **Notes**: 
> * If you use the bounding box geometry in a polar projection you'll notice a distortion due the nature of polar coordinates, if you prefer you can use the global mercator map to draw a bounding box without aparent distortion. Polygons are probably a beter idea or you can even input your own coordinates as we'll see later.
> * The calculated download size of these granules is an upper bound since Valkyrie allows us to subset the data. 


In [ ]:
v.render()

In [ ]:
# Print the granule counts from CMR for our current search params
params = v.build_params()
params

In [ ]:
# We can also query CMR to get an idea of coverages for the area we just selected
v.build_params()
granules = v.query_cmr(None)
for dataset in granules:
    size = round(sum(float(g['granule_size']) for g in granules[dataset]), 2)
    print(f'{dataset}: {len(granules[dataset])} granules found. Approx download size: {size} MB')

In [ ]:
## We can print the first record for a given dataset
dataset = 'ILATM1B'
if len(granules[dataset])>0:
    print(granules[dataset][0])

## Placing a data order

Now that we have our constraints we just need to post our order and wait for Valkyrie to fulfill it. 
We can put an order directly, in this case we are going to work on a geometry that overlaps with [Jakobshavn](https://en.wikipedia.org/wiki/Jakobshavn_Glacier) glacier in Greenland.


In [ ]:
# note that we are explicitly using the name of one of the ATM datasets
dataset = 'ILATM1B'
my_params ={
    'dataset': dataset,
    'start': '2016-01-01',
    'end': '2016-12-31',
    'bbox': '-50.221637,69.095798,-49.164471,69.29418'
}
order = v.post_valkyrie_order(my_params)
order

## Downloading the data
Let's get some coffee, some Valkyrie orders are in the Gigabytes real amd may take a little while to be processed. 
Once that your status URL says is completed we can grab the HDF5 data file using the URL on the same response!

In [ ]:
order['response'].json()

In [ ]:
import requests
my_order_status = order['response'].json()['status_url']
order_status = requests.get(my_order_status).json()
order_status

## Placing multiple data orders to Valkyrie

In [ ]:
# Or we can use the interface to put our order based in the current map selection and control values.
orders = v.post_orders()
orders